## 캐싱(Caching)

동일한 질문이 발생하게 되면, 저장된 캐시 정보를 찾는다. 그리고 답변이 있다면 GPT에 요청하지 않고 반환한다.

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 질문을 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 구성되어 있는 나라이다. 대한민국은 수도가 서울에 위치하고 있으며, 높은 경제력과 고도로 발전된 기술력을 가지고 있다. 한류 열풍으로 세계적으로 유명해진 나라로, K-pop, K-drama 등의 문화 콘텐츠가 세계적으로 인기를 끌고 있다. 조선민주주의인민공화국은 북한으로 불리며, 북한은 국제적으로 고립된 상태에 있지만, 군사적 위협을 가지고 있기도 하다. 두 국가는 과거 한반도에서 한 나라로 통일되었으나, 현재는 분단 상태가 지속되고 있다. 한반도에 위치한 한국은 역사적으로 많은 변화와 발전을 거쳐 현재의 모습을 갖추고 있다.
CPU times: user 32 ms, sys: 15 ms, total: 47 ms
Wall time: 3.13 s


## InMemoryCache
 - 휘발성이 있는 캐시 (메모리 공간을 활용해서 캐싱을 진행) 
 - 인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [5]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 나라로, 역사와 전통을 자랑하는 문화가 풍부한 나라이다. 한반도에 위치하고 있으며, 수도는 서울이다. 한국은 고대부터 현재까지 다양한 역사적 사건들을 겪어왔고, 현재는 선진화된 기술과 문화를 가지고 있는 현대화된 나라이다. 한국은 K-pop, K-drama, 한류 등의 문화 콘텐츠로 전 세계적으로 유명하며, 한글이라는 독특한 문자체계를 사용한다. 또한, 한국은 매콤한 음식과 다채로운 음식문화로 유명하며, 한국의 자연경관과 역사 유적지도 많은 관광객들이 찾는 명소이다. 현재 한국은 세계적으로 경제적으로도 성장하고 있으며, 한국인들은 열심히 일하고 노력하는 성품을 가지고 있다.
CPU times: user 278 ms, sys: 34.8 ms, total: 312 ms
Wall time: 2.86 s


- 캐시된 정보를 사용하다 보니 답변이 같다.
- 비용도 발생하지 않는다.

In [6]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 나라로, 역사와 전통을 자랑하는 문화가 풍부한 나라이다. 한반도에 위치하고 있으며, 수도는 서울이다. 한국은 고대부터 현재까지 다양한 역사적 사건들을 겪어왔고, 현재는 선진화된 기술과 문화를 가지고 있는 현대화된 나라이다. 한국은 K-pop, K-drama, 한류 등의 문화 콘텐츠로 전 세계적으로 유명하며, 한글이라는 독특한 문자체계를 사용한다. 또한, 한국은 매콤한 음식과 다채로운 음식문화로 유명하며, 한국의 자연경관과 역사 유적지도 많은 관광객들이 찾는 명소이다. 현재 한국은 세계적으로 경제적으로도 성장하고 있으며, 한국인들은 열심히 일하고 노력하는 성품을 가지고 있다.
CPU times: user 2.55 ms, sys: 251 μs, total: 2.8 ms
Wall time: 2.81 ms


## SQLite Cache

- 인메모리 캐시가 유용한 경우
    - 사용자 맞춤형 질문을 해야될 경우 (개인화)
    - 동일한 질문이라도 사용자마다 원하는 방향 혹은 취향이 달라 답변을 다르게 해야될 수 있음

In [7]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [8]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한반도 동쪽에 위치한 한국은 서술형 공화국으로서 수도는 서울이다. 한국은 한국어를 공용어로 사용하며, 국민 대다수는 불교나 기독교를 믿고 있다. 또한, 한국은 세계에서 세 번째로 큰 철강 생산국이며, 세계에서 가장 많이 수출하는 자동차 브랜드 중 하나인 현대자동차와 기아자동차가 있다. 한국은 전통적인 문화와 현대적인 기술이 조화로운 나라로서 K-pop, K-drama 등의 대중문화로 세계적인 인기를 끌고 있다. 현재는 북한과의 관계 개선을 위해 노력하고 있으며, 경제적으로도 세계에서 주요한 역할을 하고 있다.
CPU times: user 17.2 ms, sys: 3.52 ms, total: 20.7 ms
Wall time: 2.92 s
